# Processing HTML Files

We will be using **html2parquet transform**

References
- [html2parquet](https://github.com/IBM/data-prep-kit/tree/dev/transforms/language/html2parquet/python)

## Step-1: Data

We will process data that is downloaded using [1_crawl_site.ipynb](1_crawl_site.ipynb).

We have a couple of crawled HTML files in  `input` directory. 

## Step-2: Configuration

In [2]:
## All config is defined here
from my_config import MY_CONFIG

In [3]:
import os, sys
import shutil

shutil.rmtree(MY_CONFIG.OUTPUT_DIR, ignore_errors=True)
shutil.os.makedirs(MY_CONFIG.OUTPUT_DIR, exist_ok=True)

print ("✅ Cleared  output directory")

✅ Cleared  output directory


## Step-3: HTML2Parquet

Process HTML documents and extract the text in markdown format

In [4]:
from dpk_html2parquet.transform_python import Html2Parquet

x=Html2Parquet(input_folder= MY_CONFIG.INPUT_DIR, 
               output_folder= MY_CONFIG.OUTPUT_DIR, 
               data_files_to_use=['.html'],
               html2parquet_output_format= "markdown"
               ).transform()

14:29:30 INFO - html2parquet parameters are : {'output_format': <html2parquet_output_format.MARKDOWN: 'markdown'>, 'favor_precision': <html2parquet_favor_precision.TRUE: 'True'>, 'favor_recall': <html2parquet_favor_recall.TRUE: 'True'>}
14:29:30 INFO - pipeline id pipeline_id
14:29:30 INFO - code location None
14:29:30 INFO - data factory data_ is using local data access: input_folder - input output_folder - output
14:29:30 INFO - data factory data_ max_files -1, n_sample -1
14:29:30 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.html'], files to checkpoint ['.parquet']
14:29:30 INFO - orchestrator html2parquet started at 2025-03-28 14:29:30
14:29:30 INFO - Number of files is 91, source profile {'max_file_size': 0.33576393127441406, 'min_file_size': 0.10031604766845703, 'total_file_size': 11.858266830444336}
14:29:30 INFO - Completed 1 files (1.1%) in 0.004 min
14:29:30 INFO - Completed 2 files (2.2%) in 0.004 min
14

## Step-4: Inspect the Output


In [5]:
from file_utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(MY_CONFIG.OUTPUT_DIR)

print ("Output dimensions (rows x columns)= ", output_df.shape)

output_df.head(5)

## To display certain columns
#parquet_df[['column1', 'column2', 'column3']].head(5)

Output dimensions (rows x columns)=  (91, 6)


,title,document,contents,document_id,size,date_acquired
0,thealliance_ai_working-groups-hardware-enablem...,thealliance_ai_working-groups-hardware-enablem...,[Hardware Enablement Focus Area](/focus-areas/...,698eddd25c4e6e9f172a19ebec695247c0a72e6ec88c66...,1553,2025-03-28T14:29:31.461850
1,thealliance_ai_blog-open-source-ai-demo-night-...,thealliance_ai_blog-open-source-ai-demo-night-...,"On August 8th, The AI Alliance, in collaborati...",7802bb7e50653e6b21f571b28843fd9a4bcf5023eaab3a...,3151,2025-03-28T14:29:30.846980
2,thealliance_ai_working-groups-applications-and...,thealliance_ai_working-groups-applications-and...,[Applications and Tools Focus Area](/focus-are...,1aaa9d752f74d7abd233abbd8688884c99ea64f575162b...,1565,2025-03-28T14:29:31.422017
3,thealliance_ai_blog-open-innovation-day-tokyo_...,thealliance_ai_blog-open-innovation-day-tokyo_...,"Open innovation in AI software, algorithms, da...",2f82c2d26c751fcb2528eb7c9273ebf3fac4d21b842787...,1304,2025-03-28T14:29:30.839293
4,thealliance_ai_blog-ai-alliance-skills-and-edu...,thealliance_ai_blog-ai-alliance-skills-and-edu...,"By Rebekkah Hogan (Meta), Sowmya Kannan (IBM),...",a8c21ef29afc54923a30393be621693674a1ad23965998...,3615,2025-03-28T14:29:30.633874


In [6]:
output_df.iloc[0,]['title']

'thealliance_ai_working-groups-hardware-enablement_text.html'

In [7]:
output_df.iloc[0,]['document']

'thealliance_ai_working-groups-hardware-enablement_text.html'

In [8]:
## Display markdown text
print ('content length:', len(output_df.iloc[0,]['contents']), '\n')
print (output_df.iloc[0,]['contents'])


content length: 1553 

[Hardware Enablement Focus Area](/focus-areas/hardware-enablement)

# Hardware Enablement Working Group

## Co-leads

- Adam Pingel (IBM)
- Amit Sangani (Meta)

## Frequently Asked Questions (FAQ)

**How can my organization join the AI Alliance as an organizational member?**Please[send a message via our contact form](https://thealliance.ai/contact). Thanks!**How can I join as an individual contributor?**Please complete the[working group application form](https://thealliance.ai/become-a-collaborator). Thanks!**How do I get access to the AI Alliance Slack?**Once your application – as an organization or individual member – has been reviewed and approved, you will be invited to the AI Alliance Slack and receive additional instructions how to join our community.**What if I have additional questions?**[Please contact us](https://thealliance.ai/contact).

## Join the Hardware Enablement Working Group

By submitting this form, you agree that the AI Alliance will collect 

In [9]:
## display markdown in pretty format
# from IPython.display import Markdown
# display(Markdown(output_df.iloc[0,]['contents']))


## Step-5: Save the markdown

In [10]:
import os

for index, row in output_df.iterrows():
    html_file = row['document']
    base_name = os.path.splitext(os.path.basename(html_file))[0]
    md_output_file = os.path.join(MY_CONFIG.OUTPUT_DIR, base_name +  '.md')
    
    with open(md_output_file, 'w') as md_output_file_handle:
        md_output_file_handle.write (row['contents'])
# -- end loop ---       

print (f"✅ Saved {index+1} md files into '{MY_CONFIG.OUTPUT_DIR}'")

✅ Saved 91 md files into 'output'
